# Notebook 2:  Data Cleaning 

In this notebook we merge our raw data tables from Notebook 1 and clean the player salaries.
1.  **Merging**: We update our 2020 free agent data with the 2021 next-season salaries, and also merge current-season (2021-22) stats and salaries together.  This gives us two new dataframes:  df2020_merged and df2021_merged.
2.  **Cleaning salaries**: We convert salaries into numerical form.  We also normalize salaries from all years to the 2021 salary cap for consistency.

## Imports

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup, Comment
import requests
import lxml
import unicodedata
import json, pickle

## Utility Functions

In [117]:
def strip_accents_and_punctuation(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')

# (1) Merge Scraped Data

Merging 2021 data with current data

In [118]:
df2022 = pd.read_csv('./data/df2022_raw.csv')
df2023salaries = pd.read_csv('./data/dfcurrentsalaries.csv')
df2023stats = pd.read_csv('./data/dfcurrentstats.csv')
df2023urls = pd.read_csv('./data/dfplayer_to_url.csv')
df2021_merged = pd.read_csv('./data/df2021_merged.csv')
df2020_raw = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2020_raw.csv')

## a. Update df2022 with 2023 salaries

In [120]:
#df2022_merged['NameYear'] = '2022'
df2022['NameYear'] = '2022'


In [121]:
df2022_merged = pd.merge(df2022, df2023salaries, left_on="Name", right_on="Name", how="left")
df2022_merged = df2022.drop(columns=['CurrentSalary'])
df2022_merged = df2022.rename(columns = {'CurrentSalary':'Salary'})
df2022_merged = df2022.set_index('NameYear')
df2022_merged.head(3)

,Unnamed: 0.1,Unnamed: 0,Name,Pos,Age,Tm,G,GS,MP,FG,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID
NameYear,,,,,,,,,,,,,,,,,,,,,
2022,0,0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,...,11.3,14.9,0.503,18.5,-0.2,2.5,0.070,"$4,379,527",TOR,/players/a/achiupr01.html
2022,1,1,Steven Adams,C,28,MEM,76,75,26.3,2.8,...,19.6,19.9,0.560,12.0,2.0,6.8,0.163,"$12,600,000",MEM,/players/a/adamsst01.html
2022,2,2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,...,14.4,17.5,0.608,25.0,2.7,7.2,0.188,"$32,600,060",MIA,/players/a/adebaba01.html


## b. Update df2022 with 2023 salaries and player urls

In [122]:
df2022_merged = pd.merge(df2023stats, df2023salaries, left_on="Player", right_on="Name", how="left")
df2022_merged = pd.merge(df2022_merged, df2023urls, left_on="Player", right_on="Name", how="left")

df2022_merged = df2022_merged.drop(columns=['Name_x', 'Name_y','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','Rk'])
df2022_merged = df2022_merged.rename(columns = {'Player':'Name'})

df2022_merged.head(3)

,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,.485,...,11.4,16.3,.554,19.4,-0.1,2.2,.093,"$4,379,527",TOR,/players/a/achiupr01.html
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,.597,...,19.8,22.7,.564,14.6,0.7,3.4,.144,"$12,600,000",MEM,/players/a/adamsst01.html
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,.540,...,12.7,15.5,.592,25.2,2.3,7.4,.137,"$32,600,060",MIA,/players/a/adebaba01.html


## c. Update df2022 with current team and total stats

This is a little bit tricky because some players played for multiple teams.  As an example, Nickeil Alexander-Walker switched teams mid-season and thus has three data rows:

* Name    /   Team
* Nickeil Alexander-Walker  /    TOT
* Nickeil Alexander-Walker  /    NOP
* Nickeil Alexander-Walker  /    UTA

We want to keep his **season stats** from the first row (TOT) but also indicate his **current team** from the last row (UTA).  The same pattern holds for all players who played for multiple teams.

In [123]:
# Create current team name filter 
justteam = ~df2022_merged.duplicated(subset=['Name'],keep='last')
# Create total stats filter
juststats = ~df2022_merged.duplicated(subset=['Name'],keep='first')


df2022_merged = (pd.merge(df2022_merged[juststats], df2022_merged[justteam][['Name','Tm']], 
                          left_on="Name", right_on="Name", how="left")
                            .rename(columns ={'Tm_x':'TeamOrTot', 'Tm_y':'Team'}) 
         )

df2022_merged['NameYear'] = df2022_merged['Name'].apply(strip_accents_and_punctuation).str.replace(' ','_') + '_2022' 
df2022_merged = df2022_merged.set_index('NameYear')

Minor bookkeeping:
Update a few Teams with correct info (only applies to 15 players, not that important):


mask = df2022_merged.CurrentTeam.notna() & (df2022_merged.CurrentTeam != df2022_merged.Team)
df2022_merged.loc[mask,'Team'] = df2022_merged.loc[mask,'CurrentTeam']

df2022_merged = df2022_merged.drop(columns=["CurrentTeam"])
df2022_merged = df2022_merged.rename(columns= {'CurrentSalary': 'Salary'})

df2022_merged.head(5)

## Save merged data

In [124]:
df2021_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_merged.csv')
df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')

# (2) Clean salary data

## Load files

In [125]:
df2020_merged = pd.read_csv('./data/df2020_merged.csv')
df2021_merged = pd.read_csv('./data/df2021_merged.csv')

df  = pd.concat([df2020_merged, df2021_merged])
df = df.set_index('NameYear')

df2022 = pd.read_csv('./data/df2022_merged.csv')
#df2022 = df2022.set_index('NameYear')
#df2022 = df2022.drop("Player_2022")  #spurious entry

## Delete unnecessary entries

We first employ a very light common-sense filter on our entries.  To train our player value model, we wish to keep only players who: 
1.  Played in at least one game the previous year
2.  Had a salary the next year (i.e. actually received a contract in free agency)

In [126]:
df2022 = df2022[df2022['CurrentSalary'].notnull()]

In [127]:
#  1. Mask for players who played in at least one game the previous year

df['G'] = df['G'].apply(lambda s: pd.to_numeric(s, errors='coerce'))
mask1 = df['G'].isna()
df = df[~mask1]

#  2. Mask for players who had a salary the next year

mask2  = (df['NextSal'] == '0') | df['NextSal'].isna()  
df = df[~mask2]
df.sample(3)

,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,Height,Age,...,AST%,BLK%,DRB%,FTr,ORB%,STL%,TOV%,TRB%,Salary,Team
NameYear,,,,,,,,,,,,,,,,,,,,,
Moritz_Wagner_2020,Moritz Wagner,F-C,UFA,ORL,"6.9 Pts, 3.2 Reb, 1.1 Ast",ORL,/players/w/wagnemo01.html,245.0,6-11,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tyrell_Terry_2020,Tyrell Terry,G,UFA,DAL,"1.0 Pts, 0.5 Reb, 0.5 Ast",MEM,/players/t/terryty01.html,160.0,6-2,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jarrett_Allen_2020,Jarrett Allen,C-F,RFA,CLE,"12.8 Pts, 10.0 Reb, 1.7 Ast",CLE,/players/a/allenja01.html,243.0,6-11,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean salries and normalize by salary cap

Since we are looking at data for multiple years, it is important to normalize by salary cap for apples-to-apples comparisons in our model.  We chose the 2023 salary cap for simplicity.

In [128]:
def clean_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
    '''
    if isinstance(sal, float):
        return 0
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('(TW)','').replace('Minimum','0')
            #elem =  float(elem)
            newlist.append(float(elem))
        except:
            continue
            
    salary = sum(newlist)/1000000
    
    return salary

# Salary Cap Data taken from spotrac.com/nba/cba
salarycap = {2016:  94.143, 2017: 99.093,   2018: 101.869,  2019:  109.14,   
             2020:  109.14,    2021: 112.414,   2022: 123.655,  2023: 136.021}   

def clean_and_norm_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
        Then normalizes by salary cap (in 2023 salary cap dollars)
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    salary = salary*salarycap[2023]/salarycap[year]
        
    return salary

for index, row in df.iterrows():
    
    prevyear = df2022_merged.loc[index, 'PrevYear']
    nextyear = df2022_merged.loc[index, 'PrevYear'] + 1 

    prevsal = df2022_merged.loc[index, 'PrevSal']
    nextsal = df2022_merged.loc[index, 'NextSal']
    
    df2022_merged.loc[index, 'PrevSalClean'] = clean_salary(prevsal, prevyear)
    df2022_merged.loc[index, 'NextSalClean'] = clean_salary(nextsal, nextyear)
    df2022_merged.loc[index, 'PrevSalNorm'] = clean_and_norm_salary(prevsal, prevyear)
    df2022_merged.loc[index, 'NextSalNorm'] = clean_and_norm_salary(nextsal, nextyear)
   
cols = ['NextSal', 'NextSalClean','NextSalNorm']
df2022_merged[cols].sample(8)

KeyError: 'PrevYear'

In [129]:
for index, row in df2022.iterrows():
    
    salary = df2022.loc[index, 'CurrentSalary']
    try:
        df2022.loc[index, 'SalClean'] = clean_salary(CurrentSalary, 2023)
    except:
        df2022.loc[index, 'SalClean'] = np.nan
        
    try:
        df2022.loc[index, 'SalNorm'] = norm_salary(CurrentSalary, 2023)
    except:
        df2022.loc[index, 'SalNorm'] = np.nan

df2022.sample(3)

,NameYear,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,...,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID,Team,SalClean,SalNorm
49,Devin_Booker_2022,Devin Booker,SG,26,PHO,53,53,34.6,9.9,20.1,...,31.8,2.9,6.0,.157,"$36,016,200",PHO,/players/b/bookede01.html,PHO,NaN,NaN
220,Caleb_Houstan_2022,Caleb Houstan,SF,20,ORL,51,4,15.9,1.3,3.6,...,11.0,-0.4,0.7,.039,"$2,000,000",ORL,/players/h/houstca01.html,ORL,NaN,NaN
397,Jason_Preston_2022,Jason Preston,PG,23,LAC,14,0,8.9,1.3,2.9,...,18.3,0.0,0.1,.021,"$1,836,096",LAC,/players/p/prestja01.html,LAC,NaN,NaN


## Save cleaned data

df.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/cleaned_pastyears.csv')
df2021.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/cleaned_currentyear.csv')



In [41]:
#cleaning error somewhere, manually cleaned up and saved in last code box

#df.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_pastyears.csv')
#df2022.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_currentyear.csv')


In [130]:
df2022_merged = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')

#df2022_merged = df2022_merged[~df2022_merged['Name'].str.contains('Player')]

df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')

df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_currentyear.csv')


In [4]:
df2022_merged = df2022_merged.drop("Unnamed: 0", axis=1)

df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')

df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_currentyear.csv')